# Desafio Kinea


## Importando e definindo constantes globais

In [1]:
import requests
import base64 as b64
import json
import pandas as pd
from typing import Dict, List
import csv
from dateutil.relativedelta import relativedelta

In [2]:
import logging
logging.level=logging.INFO

In [3]:
from datetime import date, datetime
from cal import AmbimaCalendar

In [4]:
client_id = "9r2j5fK3DntV"
client_secret = "ufaORoKwRRmw"

## Funções básicas

In [54]:
def dias_uteis(start_date, end_date) -> int:
  """O formato de data é date(AAAA, MM, DD), como date(2022, 2, 4) para 4 de fev de 2022"""
  interval = AmbimaCalendar().get_working_days_delta(start_date, end_date)+1
  return interval

In [6]:
def b64encode(data:str) -> str:
  "Codifica uma string em base64"
  encodedBytes = b64.b64encode(data.encode("utf-8"))
  encodedStr = str(encodedBytes, "utf-8")
  return encodedStr

In [7]:
def check_error(acquired_value, expected_value):
  err = abs((acquired_value-expected_value)/expected_value)*100
  err_str = f'{err:0.3}%'
  return err_str

## Interface com API Ambima

In [30]:
def get_access_token(_id:str, secret:str) -> requests.models.Response:
  """Recebe id e secret de um dado app e retorna o toke de acesso para aquele app"""
  address = "https://api.anbima.com.br/oauth/access-token?"
  credentials = b64encode(f"{_id}:{secret}")
  body = {"grant_type": "client_credentials"}
  headers = {
    "Authorization": f"Basic {credentials}",
    "Content-Type": "application/json"
  }
  logging.debug(f"Auth headers: {headers}")
  res = requests.post(address, headers=headers, json=body)
  logging.info(f"Auth res status: {res.status_code}")
  logging.debug(f"Auth res text: {res.text}")
  return res


In [31]:
def get_data(_id:str, token:str, address:str) -> requests.models.Response:
  headers = {
    "Content-Type": "application/json",
    "access_token": token,
    "client_id": _id}
  logging.debug(f"Auth headers: {headers}")
  res = requests.get(address, headers=headers)
  logging.info(f"Auth res status: {res.status_code}")
  if res.status_code == 401:
    new_token = get_access_token(_id, client_secret)
    get_data(_id, new_token, address)
  logging.debug(f"Auth res text: {res.text}")
  return res

In [32]:
auth_res = get_access_token(client_id, client_secret)
access_token = json.loads(auth_res.content)["access_token"]
logging.info(f"Access Token: {access_token}")

In [33]:
address_base = "https://api-sandbox.anbima.com.br"
address_mercado_secundario_TPF = f"{address_base}/feed/precos-indices/v1/titulos-publicos/mercado-secundario-TPF"
addres_estimativa_selic = f'{address_base}/feed/precos-indices/v1/titulos-publicos/estimativa-selic'
adress_vna = f"{address_base}/feed/precos-indices/v1/titulos-publicos/vna"
adress_ipca_igpm = f"{address_base}/feed/precos-indices/v1/titulos-publicos/projecoes"

data_res = get_data(client_id, access_token, address_mercado_secundario_TPF)
data_list = json.loads(data_res.content)
logging.info(data_list)

In [34]:
data_res = get_data(client_id, access_token, adress_vna)
data_vna = json.loads(data_res.content)
data_vna

[{'data_referencia': '2017-10-20',
  'titulos': [{'tipo_titulo': 'NTN-C',
    'codigo_selic': '770100',
    'index': 0.37,
    'tipo_correcao': 'P',
    'data_validade': '2017-10-13',
    'vna': 3531.425676},
   {'tipo_titulo': 'LFT',
    'codigo_selic': '210100',
    'index': 8.15,
    'tipo_correcao': 'O',
    'data_validade': '2017-10-20',
    'vna': 9162.202647},
   {'tipo_titulo': 'NTN-B',
    'codigo_selic': '760100',
    'index': 0.45,
    'tipo_correcao': 'P',
    'data_validade': '2017-10-17',
    'vna': 3013.085961},
   {'tipo_titulo': 'NTN-B',
    'codigo_selic': '760199',
    'index': 0.45,
    'tipo_correcao': 'P',
    'data_validade': '2017-10-17',
    'vna': 3013.085961}]}]

In [47]:
# [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='210100'][0]

9162.202647

In [12]:
def save_to_csv(filedata:List[Dict], filename:str):
  """Take a list of dicts and save as an .csv with the given name"""
  keys = filedata[0].keys()
  with open('exports\\'+filename, 'w', newline='') as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writeheader()
      dict_writer.writerows(filedata)

In [13]:
def clean_mercado_secundario(list_to_clean:List[Dict]):
  keep_keys = ['tipo_titulo','data_referencia','data_vencimento','data_base','taxa_indicativa','pu']
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [14]:
clean_mercado = clean_mercado_secundario(data_list)

## Cálculo de PUs

In [15]:
def get_dias_cupom(start_date, end_date):
  """Retorna uma listagem de cupons entre duas datas, indo da data final até a inicial, de 6 em 6 meses"""
  pgto_date_list = list()
  pgto_date = end_date
  # Gera da lista de datas a partir da data final, em incrementos de 6 meses
  while pgto_date > start_date:
    pgto_date_list.append(pgto_date)
    pgto_date -= relativedelta(months=6)
  # Inverte a ordem da lista para crescente
  pgto_date_list=pgto_date_list[::-1]
  
  return pgto_date_list

LTN

In [16]:
def calcular_ltn(taxa_anual, data_liqui, data_venc, valor_face=1000, expected_pu=0, debug=False):
  du = dias_uteis(data_liqui, data_venc)
  pu = valor_face/((1+taxa_anual)**(du/252))
  pm = (data_venc - data_liqui).days
  duration = du
  return pu, duration, pm

In [17]:
# calcular_ltn(0.1212, date(2022,3,21), date(2025,1,1))
calcular_ltn(0.0728, date(2017,10,20), date(2018,1,1))

(986.7040298582051, 48, 73)

NTN-F

In [23]:
def calcular_ntn_f(taxa, data_liqui, data_venc):
  lista_datas = get_dias_cupom(data_liqui, data_venc)
  fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
  fluxo_pgto['cupom_semestral'] = [(1.1**(1/2)-1)*1000]*len(lista_datas)
  fluxo_pgto['cupom_semestral'].iloc[-1] = fluxo_pgto['cupom_semestral'].iloc[-1] + 1000
  fluxo_pgto['dias_uteis'] = fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_liqui, x))
  fluxo_pgto['dias_corridos'] = fluxo_pgto['data_pgto'].apply(lambda x: (x - data_liqui).days)
  fluxo_pgto['vp_pgto'] = fluxo_pgto.apply(lambda x: (x['cupom_semestral'])/((1+taxa)**(x['dias_uteis']/252)), axis=1)

  pu = fluxo_pgto['vp_pgto'].sum()
  duration = (fluxo_pgto['vp_pgto']*fluxo_pgto['dias_uteis']).sum()/pu
  pm = (fluxo_pgto['cupom_semestral']*fluxo_pgto['dias_corridos']).sum()/fluxo_pgto['cupom_semestral'].sum()

  return pu, duration, pm


In [25]:
calcular_ntn_f(0.122, date(2022,3,21), date(2033,1,1))

(897.1143129880454, 1625.9087871528416, 2945.5450927928014)

NTN-B

In [77]:
def calcular_ntnb(taxa, data_liqui, data_venc):
  # d15_base = data_liqui.replace(day=15)
  # d15_proj = d15_base + relativedelta(months=1)
  
  # VNA deveria puxar dados de um banco, porém ainda não existe esse banco e o API tem só uma data em sandbox
  vna = [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='210100'][0]

  lista_datas = get_dias_cupom(data_liqui, data_venc)
  fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
  fluxo_pgto['fluxo'] = [(1.06**(1/2)-1)*1000]*len(lista_datas)
  fluxo_pgto['fluxo'].iloc[-1] = fluxo_pgto['fluxo'].iloc[-1] + 1000
  fluxo_pgto['dias_uteis'] = fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_liqui, x))
  fluxo_pgto['dias_corridos'] = fluxo_pgto['data_pgto'].apply(lambda x: (x - data_liqui).days)
  fluxo_pgto['vf_fluxo_taxa'] = fluxo_pgto.apply(lambda x: ((1+taxa)**(x['dias_uteis']/252)), axis=1)
  fluxo_pgto['cotacao'] = fluxo_pgto.apply(lambda x: x['fluxo']/x['vf_fluxo_taxa'], axis=1)
  
  fluxo_pgto['cupom_semestral'] = fluxo_pgto['fluxo']*vna
  fluxo_pgto['vp_pgto'] = fluxo_pgto['cupom_semestral']/fluxo_pgto['vf_fluxo_taxa']

  pu = fluxo_pgto['cotacao'].sum()*vna
  duration = (fluxo_pgto['vp_pgto']*fluxo_pgto['dias_uteis']).sum()/fluxo_pgto['vp_pgto'].sum()
  pm = (fluxo_pgto['cupom_semestral']*fluxo_pgto['dias_corridos']).sum()/fluxo_pgto['cupom_semestral'].sum()
  
  return pu, duration, pm

In [78]:
calcular_ntnb(0.0576, date(2022,3,21), date(2045,5,15))

# print(f'ntn_b_princ_pu: {ntn_b_princ_pu} | ntn_b_princ_prazo_medio: {ntn_b_princ_prazo_medio} | ntn_b_princ_duration: {ntn_b_princ_duration}')
# print(f'ntn_b_princ_pu: {ntn_b_princ_pu}')
# print(f'Erro ntn_b_princ_pu: {check_error(ntn_b_princ_pu, 1059.153)}')

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


(9642452.257578263, 3218.096109513264, 6013.817072266434)

LFT

In [27]:
def calcular_lft(taxa_anual, data_liqui, data_venc, vna, valor_face=1000):
  pu = calcular_ltn(taxa_anual, data_liqui, data_venc, vna)
  return pu

In [28]:
calcular_lft(0.001193, date(2022,5,12), date(2022,9,1), 11648.93567)

(11644.527330634757, 80, 112)

## Cálculo Geral

In [73]:
for titulo in clean_mercado:
  data_liqui=datetime.strptime(titulo['data_referencia'], "%Y-%m-%d").date()
  data_venc=datetime.strptime(titulo['data_vencimento'], "%Y-%m-%d").date()

  if titulo['tipo_titulo'] == 'LTN':
    pu_calc = calcular_ltn(
      taxa_anual=titulo['taxa_indicativa']/100,
      data_liqui=data_liqui,
      data_venc=data_venc)
    titulo['pu_calc']=pu_calc

  elif titulo['tipo_titulo'] == 'NTN-B':
    pu_calc, pm, dur = calcular_ntnb(
      data_liqui=data_liqui,
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      # taxa_cupom=titulo['taxa_indicativa']/100, Esse deve ser um valor fixo para o NTN-B. Toda taxa cupom é fixa para um tipo de titulo
      # infla_proj=0.0094
      )
    titulo['pu_calc']=pu_calc
    titulo['prazo_medio']=pm
    titulo['duration']=dur



SyntaxError: invalid syntax (3466584874.py, line 24)

In [74]:
t =[x for x in clean_mercado if x['tipo_titulo'] in ['LTN','NTN-B']]

In [76]:
for x in t: print(x['tipo_titulo'], x['pu'], x['pu_calc'])

LTN 986.696496 (986.6964968762362, 48, 73)
LTN 970.982414 (970.9824147548106, 109, 163)
LTN 954.999495 (954.9994953886635, 172, 254)
NTN-B 3127.950824 9510536.50904366
LTN 938.119221 (937.8653345011708, 237, 346)
LTN 920.958993 (920.958993577912, 298, 438)
LTN 902.67683 (902.4194136427137, 360, 528)
NTN-B 3238.274966 9845962.943987137
LTN 883.262368 (883.0019750547143, 422, 619)
LTN 861.87865 (861.6156307707371, 488, 711)
LTN 842.202192 (842.2021922041295, 551, 803)
LTN 803.072295 (802.8110295218422, 675, 985)
NTN-B 3227.183797 9813087.214232948
NTN-B 3287.138181 9995428.955494266
LTN 726.164401 (725.9132482200565, 926, 1350)
NTN-B 3258.531943 9907054.202601187
NTN-B 3254.422094 9894496.944431322
NTN-B 3321.33068 10098109.444787432
NTN-B 3318.513445 10089377.309074169
NTN-B 3329.224966 10123073.802905532
NTN-B 3369.672632 10244897.680772386
NTN-B 3456.98885 10510816.50581708
NTN-B 3463.102401 10529047.221082088
NTN-B 3488.327957 10606282.537430994
NTN-B 3487.035343 10601850.006787786
N